In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
drive_path = "/content/drive/My Drive/CSCI-576"

In [ ]:
def read_image(pathname):
    img = cv2.imread(pathname, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"File not found: {pathname}")
    assert img.dtype == np.uint8
    return img.astype(np.float32) / 255.0

In [ ]:
def write_image(M, pathname):
    if M.ndim != 2:
        raise ValueError("Matrix must be 2-dimensional")
    img = np.clip(M, 0.0, 1.0) * 255.0
    img = img.astype(np.uint8)
    if not cv2.imwrite(pathname, img): #, cv2.IMWRITE_JPEG_QUALITY, 100
        raise IOError(f"Could not write image to {pathname}")

In [ ]:
def display_image(M, title):
    if M.ndim != 2:
        raise ValueError("Matrix must be 2-dimensional")
    img = np.clip(M, 0.0, 1.0)
    plt.figure(figsize=(img.shape[0] / 64, img.shape[1] / 64), dpi=128)
    plt.imshow(img, cmap='gray', vmin=0.0, vmax=1.0)
    if title: plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
image_names = [
    "grayMan"
  , "grayMan_gaussian"
  , "grayMan_saltandpepper"
  , "Q4_original"
  , "Q4_corrupted"
]
for name in image_names:
    M = read_image(f"{drive_path}/{name}.png")
    print(M.shape)

In [ ]:
# Question 1
M = read_image(f"{drive_path}/grayMan.png")
ROWS, COLUMNS = M.shape
R = -1
C = -1
while R < 0 or R > ROWS:
    try: R = int(input(f"Number of rows up to {ROWS}:"))
    except: pass
while C < 0 or C > COLUMNS:
    try: C = int(input(f"Number of columns up to {COLUMNS}: "))
    except: pass
rows = np.random.choice(np.arange(0, ROWS), size=R, replace=False)
columns = np.random.choice(np.arange(0, COLUMNS), size=C, replace=False)
S = M
S = np.delete(S, rows, axis=0)
S = np.delete(S, columns, axis=1)
display_image(M, f"Original {M.shape[0]}x{M.shape[1]}")
display_image(S, f"Sliced Up! {S.shape[0]}x{S.shape[1]}")
write_image(S, f"{drive_path}/grayMan.jpg")

In [ ]:
# Question 2
avg_kernel = np.array([[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]])
laplacian_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
names = {"grayMan_gaussian": 'g', "grayMan_saltandpepper": 'sb'}
for name in names:
    original = read_image(f"{drive_path}/{name}.png")
    ROWS, COLUMNS = original.shape
#   ------------------------------------------------------------------------------------------------
    avg_smooth = np.empty((ROWS-2, COLUMNS-2))
    for r in range(1, ROWS - 1):
        for c in range(1, COLUMNS - 1):
            avg_smooth[r-1, c-1] = np.sum(original[r-1:r+2, c-1:c+2] * avg_kernel)
#   ------------------------------------------------------------------------------------------------
    avg_laplacian = np.empty((ROWS-4, COLUMNS-4))
    for r in range(1, ROWS - 3):
        for c in range(1, COLUMNS - 3):
            avg_laplacian[r-1, c-1] = np.sum(avg_smooth[r-1:r+2, c-1:c+2] * laplacian_kernel)
#   ------------------------------------------------------------------------------------------------
    med_smooth = np.empty((ROWS-2, COLUMNS-2))
    for r in range(1, ROWS - 1):
        for c in range(1, COLUMNS - 1):
            med_smooth[r-1, c-1] = np.median(original[r-1:r+2, c-1:c+2])
#   ------------------------------------------------------------------------------------------------
    med_laplacian = np.empty((ROWS-4, COLUMNS-4))
    for r in range(1, ROWS - 3):
        for c in range(1, COLUMNS - 3):
            med_laplacian[r-1, c-1] = np.sum(med_smooth[r-1:r+2, c-1:c+2] * laplacian_kernel)
#   ------------------------------------------------------------------------------------------------
#   display_image(original,       f"Original {name} {ROWS}x{COLUMNS}")
#   display_image(avg_smooth,     f"Avg smoothing {name} {ROWS-2}x{COLUMNS-2}")
#   display_image(avg_laplacian,  f"Avg smooth+laplacian {name} {ROWS-4}x{COLUMNS-4}")
#   display_image(med_smooth,     f"Median smoothing {name} {ROWS-2}x{COLUMNS-2}")
#   display_image(med_laplacian,  f"Median smooth+laplacian {name} {ROWS-4}x{COLUMNS-4}")
#   ------------------------------------------------------------------------------------------------
    write_image(avg_smooth,       f"{drive_path}/fnA{names[name]}.jpg")
    write_image(avg_laplacian,    f"{drive_path}/fnA{names[name]}L.jpg")
    write_image(med_smooth,       f"{drive_path}/fnM{names[name]}.jpg")
    write_image(med_laplacian,    f"{drive_path}/fnM{names[name]}L.jpg")